# INF2 TP6

# Chazelas Clément, Mantion Nathan

## Code

In [ ]:
import sqlite3 as sq
import pandas as pd
from tkinter import *
 
 
#fonction pour ajouter un nouveau logeur dans la bdd
def new_logeur(nom,prenom,numero_rue,nom_rue,code_postal,ville):
    connection = sq.connect("alesc.db") #connection à la base de données
    curseur = connection.cursor() #création d'un curseur
    query = f"""
            INSERT INTO logeur (nom, prenom, numero_rue, nom_rue, code_postal, ville)
            VALUES
            ('{nom}','{prenom}','{numero_rue}','{nom_rue}','{code_postal}','{ville}');
            """ #requête sql permettant d'ajouter un nouveau logeur
    curseur.execute(query) #exécution de la requête
    connection.commit() #enregistrement de la requête
    curseur.close() #fermeture de connection avec la bdd
    connection.close()
 
#fonction pour ajouter un nouveau logement dans la bdd
def new_log(numero_rue,nom_rue,code_postal,ville,label,nom,prenom,type_logement):
    #recherche en amont du logeur pour obtenir son id et l'ajouter dans la base
    connection=sq.connect("alesc.db")
    curseur = connection.cursor()
    query = f"""
            SELECT id_logeur FROM logeur WHERE prenom='{prenom}' AND nom='{nom}';
            """
    curseur.execute(query)
    id_logeur = curseur.fetchall()[0][0]
    curseur.close()
    #ajout du logement dans la db
    curseur = connection.cursor()
    query= f"""
            INSERT INTO logement (id_logeur,numero_rue,nom_rue,ville,code_postal,type,label)
            VALUES
            ('{id_logeur}','{numero_rue}','{nom_rue}','{ville}','{code_postal}','{type_logement}','{label}');
            """
    curseur.execute(query)
    connection.commit()
    curseur.close()
    connection.close()
 
#fonction pour ajouter un étudiant dans la bdd
def new_etu(nom,prenom,semestre,numero_rue,nom_rue,code_postal):
    #recherche en amont de l'id du logement habité en fonction de l'adresse
    connection=sq.connect("alesc.db")
    curseur = connection.cursor()
    query = f"SELECT id_logement FROM logement WHERE (numero_rue={int(numero_rue)} AND nom_rue='{nom_rue}' AND code_postal={int(code_postal)});"
    curseur.execute(query)
    id_logement = curseur.fetchall()[0][0]
    curseur.close
    #ajout de l'étudiant dans la db
    curseur = connection.cursor()
    query= f"""
            INSERT INTO etudiant (nom,prenom,semestre,id_logement)
            VALUES
            ('{nom}','{prenom}','{semestre}','{id_logement}');
            """
    curseur.execute(query)
    connection.commit()
    curseur.close()
    connection.close()
 
#fonction pour retrouver un étudiant dans la bdd en fonction de son logement
def get_etudiants(id_logement):
    connection=sq.connect("alesc.db") #connection à la base
    curseur=connection.cursor() #création du curseur
    query = f"""
            SELECT * FROM etudiant WHERE id_logement={int(id_logement)};
            """ #requête pour retrouver les étudiants en fonction de leur logement
    curseur.execute(query) #exécution de la requete
    etus = curseur.fetchall()
    curseur.close()
    etus_clean = [] #création de la liste qui sera retourné par la fonction
    #Boucle pour créer des dictionnaires, un dictionnaire correspondant à un étudiant
    for etu in etus:
        etu_clean={} #création du dictionnaire vide
        etu_clean["nom"]=etu[0] #ajout du nom
        etu_clean["prenom"]=etu[1] #ajout du prenom
        etu_clean["semestre"]=etu[2] #ajout du dictionnaire
        etus_clean.append(etu_clean) #ajout de l'étudiant dans la liste des étudiant du logement
    connection.close() #fermeture de la connection avec la bdd
    return etus_clean #on retourne la liste obtenue
 
#fonction pour retrouver le logements d'un logeur ainsi que ses occupants
def get_logements(nom_logeur,prenom_logeur):
    connection=sq.connect("alesc.db") #onnection à la bdd
    curseur = connection.cursor() #création du curseur
    query = f"""
            SELECT id_logeur FROM logeur WHERE prenom='{prenom_logeur}' AND nom='{nom_logeur}';
            """ #requête SQL pour retrouver l'id du logeur qui nous permettra de retrouver les logements li appartenant par la suite
    curseur.execute(query) #exécution de la requête
    id_logeur = curseur.fetchall()[0][0] #On est censés trouver un unique résultat, contenant une seule valeur que l'on récupère
    curseur.close()
    curseur=connection.cursor()
    query = f"""
            SELECT * FROM logement WHERE id_logeur={int(id_logeur)};
            """ #Requête pour retrouver tous les logements à partir de l'id_logeur
    curseur.execute(query) #exécution de la requete
    logements = curseur.fetchall()
    curseur.close()
    logements_clean = [] #création de la liste contenant toutes les informations de chaque logements appartenant au logeur
    #Boucle pour créer des dictionnaires, un dictionnaire correspondant à un logement
    for logement in logements:
        logement_clean={} #création du dictionnaire vide
        logement_clean["numero_rue"]=logement[2] #ajout du numéro de rue du logement
        logement_clean["nom_rue"]=logement[3] #ajout du nom de la rue
        logement_clean["ville"]=logement[4] #ajout du nom de la ville
        logement_clean["code_postal"]=logement[5] #ajout du code postal
        logement_clean["type"]=logement[6] #ajout ajout du type de logement
        logement_clean["label"]="*"*int(logement[7]) #ajout ajout du label sous forme de nombre de caractère "*" et non d'entier pour une meilleure visualisation par l'utilisateur
        logement_clean["etu"]=get_etudiants(logement[0]) #ajout des information des étudiants habitant dans le logement a partir de la fonction définie plu haut
        logements_clean.append(logement_clean) #ajout du logement dans la liste des logement du logeur
    connection.close() #fermeture de la connection avfec la bdd
    return logements_clean #on retourne la liste obtenue
 
 
 
if __name__ == "__main__":
    df = pd.read_excel("logeurs.xlsx")
    for logeur in df.values.tolist():
        new_logeur(logeur[0], logeur[1], logeur[2], logeur[3], logeur[4], logeur[5])
 
    df = pd.read_excel("logements.xlsx")
    for logement in df.values.tolist():
        new_log(logement[0], logement[1], logement[2], logement[3], logement[4], logement[5], logement[6], logement[7])
 
 
    df = pd.read_excel("etudiants.xlsx")
    for etudiant in df.values.tolist():
        new_etu(etudiant[0], etudiant[1], etudiant[2], etudiant[3], etudiant[4], etudiant[5])
 
 
    nom_logeur= input("Nom du logeur : ")
    prenom_logeur = input("Prenom du logeur : ")
    logements = get_logements(nom_logeur, prenom_logeur)
    num=1
    for logement in logements:
        print(f"Logement {num}")
        print(f"{logement['numero_rue']} {logement['nom_rue']} {logement['code_postal']} {logement['ville']} {logement['label']} {logement['type']}")
        for etu in logement["etu"]:
            print(f"Nom etudiant : {etu['nom']} {etu['prenom']}")
    num+=1
 
class Fenetre(Tk) :
    def __init__(self) :
        Tk.__init__(self)
        self.title('TP6')
        self.prenomentry = StringVar() #les deux variables utilisées pour appeler la fonction parcourant la base de données
        self.nomentry = StringVar()
        self.prenom()
        self.prenom_etiquette()
        self.nom()
        self.nom_etiquette()
        self.valider()
        self.reinitialiser()
        self.Quitter()
       
 
 
 
    def prenom(self) :
        self.entrée_prénom = Entry(self,textvariable=self.prenomentry)  #On crée l'entry du prénom
        self.entrée_prénom.grid(row = 1, column = 10, columnspan = 3, padx = 10, pady =15)
 
    def prenom_etiquette(self) :
        self.etiquette = Label(self, text="Prénom du logeur : ",bg="yellow")   #On affiche "Prénom du logeur" sur un fond jaunedevant le champ de saisie du prénom 
        self.etiquette.grid(row = 1, column = 1, columnspan = 3, padx = 10, pady =15)
 
    def nom(self) :
        self.entrée_nom = Entry(self, textvariable=self.nomentry)   #On crée l'entry du nom
        self.entrée_nom.grid(row = 0, column = 10, columnspan = 3, padx = 10, pady =15)
 
    def nom_etiquette(self) :
        self.etiquette = Label(self, text="Nom du logeur : ",bg="yellow")    #On affiche "Nom du logeur" sur un fond jaunedevant le champ de saisie du nom
        self.etiquette.grid(row = 0, column = 1, columnspan = 3, padx = 10, pady =15)
 
    def valider(self) :
        self.bouton = Button(self, text="Valider", command=self.affiche)  #On crée le bouton valider qui executera la fonction "affiche" quand pressé
        self.bouton.grid(row = 3, column = 1, padx = 10, pady =15)
 
    def reinitialiser(self) :
        self.rei = Button(self, text="Reinitaliser", command=self.effacer)    #On crée le bouton reinitialiser qui executera la fonction "effacer" quand pressé
        self.rei.grid(row = 3, column = 3, padx = 10, pady =15)
 
    def Quitter(self) :
        self.rei = Button(self, text="Quitter", command=self.quitter)    #On crée le bouton Quitter qui executera la fonction "quitter" quand pressé
        self.rei.grid(row = 3, column = 4, padx = 10, pady =15)
 
    def quitter(self) : #Fonction de destruction de la fenêtre
        self.destroy()
 
    def effacer(self) : #Fonction de vide des entrées
        self.prenomentry.set("")
        self.nomentry.set("")
         
    def affiche(self) :
        nom = self.nomentry.get() #On récupère les valeurs entrées pas l'utilisateur
        prenom = self.prenomentry.get() #On récupère les valeurs entrées par l'utilisateur
        logements = get_logements(nom, prenom) #On utlise la fonction définie plus haut pour récupérer les information des logements liées au logeur
 
        x = 1    #On crée deux variables qui vont nous aider pour l'affichage du numéro du logement et le placement des différenrts widgets crées dans les boucles suivantes
        y = 0
 
 
        for logement in logements :    #On traite les informations des logements logement par logement
            self.y = Label(self, text = f'logement {y+1}',bg="red")    #On affiche le numéro du logement sur un fond rouge
            self.y.grid(row = 3 + x, column = 1)
 
            #On affiche l'adresse ainsi que le code postal, la ville, le label ainsi que le type du logement :
            self.w = Label(self, text = f'{logement["numero_rue"]} {logement["nom_rue"]} {logement["code_postal"]} {logement["ville"]} {logement["label"]} {logement["type"]}')    
            self.w.grid(row = 4 + x, column = 1)
         
            for etu in logement["etu"] :    #On affiche les infos de l'étudiant/des étudiants qui occupent le logement étudiant par étudiant
                self.a = Label(self, text="Nom de l'étudiant")
                self.a.grid(row = 5 + x, column = 1, padx = 5, pady =7)
                self.b = Label(self, text=f'{etu["nom"]} {etu["prenom"]}')
                self.b.grid(row = 5 + x, column = 2, columnspan = 3, padx = 10, pady =15)
                x = x + 2
            x = x + 2
            y = y + 1
 
fenetre = Fenetre()
fenetre.mainloop()


Création de la base de données :
Pour créer la base de données, nous avons cherché à trouver quels sont les liens à sens uniques. Les étudiants vivent dans un seul logement, les logements n’appartiennent qu’à un seul logeur. Par conséquent, il faut que l’étudiant ait une donnée lié au logement et logement au logeur. Nos clés primaires et étrangères étaient donc trouvées. On ajoute au logeur un identifiant unique, clé primaire. Cet identifiant sera une clé étrangère dans le logement, ce logement aura lui aussi un identifiant unique en clé primaire qui sera utilisé dans la table des étudiants. Cela nous permet de lier en étudiant à un logement avec une donnée (l’ID du logement) plutôt qu’une suite de donnés (une adresse avec numéro de rue, nom de rue, etc…). De même on lie un logement à un logeur avec une seule donnée plutôt que deux (nom et prénom). Ensuite chaque table à ses données propres. Par exemple le semestre pour étudiant ou le type pour un logement.
Voir schéma

Code :
Pour être plus efficaces, nous avons divisé le travail de programmation en deux parties, back-end et front-end. C'est-à-dire le code traitant les données de la base et le code affichant les résultats de recherche.
Concernant le code back-end, il s’est divisé en 5 fonctions. 3 fonctions pour transformer les données des tableurs excel en entrées sur la base SQL (une fonction par table) et 2 pour retrouver les données de logements en fonction des demandes de l’utilisateur.
Pour les fonctions d’entrées, les données en entrée de la fonction sont ajoutées dans la table correspondante avec du code SQL exécuté avec le module sqlite3. Lorsque cela est nécessaire, on utilise aussi une requête “SELECT” avant l’insertion afin de retrouver les clés permettant de faire le lien entre les tables. Par exemple, avant d’entrer un logement, on utilise une requête pour retrouver l’identifiant du logeur lié au logement.
Les deux fonctions de recherche de données sont toutes les deux liées.
La fonction “get_logements” permet de retrouver les logements d’un logeur. En premier lieu, on retrouve l’identifiant du logeur pour en retrouver les logements. Une fois ces logements retrouvés, sqlite3 renvoie une liste de tuple, on utilise alors une boucle for pour retrouver les étudiants liés à l’identifiant du logement avec la fonction “get_etu”. Cette fonction nous renvoie une liste de dictionnaires, chaque dictionnaire correspondant à un étudiant. Les données des étudiants couplées à celles du logement nous permettent ensuite de construire une liste de dictionnaires, chaque dictionnaire correspondant à un logement. Parmi les attributs de ce dictionnaire l’attribut “etu” correspond à la liste renvoyée par la fonction “get_etu”.
Cette liste de dictionnaires sera ensuite traitée dans la partie front-end du code.

Tkinter :
On a tout d'abord crée les premiers widgets, soit les champs d'entré du nom et du prénom du logeur, leurs étiquettes premettant d'afficher ce que l'on demande à l'utilisateur respectives, ainsi que le bouton valider qui déclenchera l'affichage des logements du logeurs avec leurs info, le bouton reinitialiser qui remet à 0 la page, et le bouton quitter qui ferme la page. On défini ensuite la fonction affiche. Pour cela on va tout d'abord récupérer le nom et le prénom entrés par l'utilsateur puis on récupère les informations des logements de l'utilisateur à l'aide de la fonction "get_logements" que nous avons définis plus tôt. On fait ensuite une boucle dans une boucle pour traiter les logements et leurs occupant un à un . C'est à dire que pour chaque logement on affichera ses infos puis celles de ses occupants. On utilisera les variables x et y pour respectivement placer les widgets sur notre grid et afficher le numéro du logement.